In [1]:
import os  # For fetching environment variables
import sys

# Add the parent directory to the sys.path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import time
import datetime
import math
import pandas as pd
import numpy as np
from aave_client import AaveClient

/Users/cleme/cleyfe/Aave_Functions/aave_functions/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Settings

In [3]:

start_time=time.time()

token=['USDC', 'USDT', 'DAI']

general_path=os.path.abspath(os.path.dirname(os.getcwd()))
result_template_path=general_path+'\\Result\\'+'\\'+f'{token} - Result Template.xlsx'

# Initial Token Amount
initial_token_amt = 100

# Info on the AAVE web page
# https://app.aave.com/#/deposit
ltv_threshold=0.93
max_ltv=0.9
ltv_list=[0.9]

RPC = "POLYGON"


# Initialize Client

In [4]:
if RPC == "POLYGON":
    RPC_URL = os.getenv("POLYGON_RPC_URL")
elif RPC == "ETHEREUM":
    RPC_URL = os.getenv("MAINNET_RPC_URL")
elif RPC == "ARBITRUM":
    RPC_URL = os.getenv("ARBITRUM_RPC_URL")
elif RPC == "KOVAN":
    RPC_URL = os.getenv("KOVAN_RPC_URL")
elif RPC == "MUMBAI":
    RPC_URL = os.getenv("MUMBAI_RPC_URL")
else:
    RPC_URL = None  # or handle the case when the RPC does not match any known value

# Define a dictionary to map RPC to the appropriate keyword argument
rpc_url_args = {
    "POLYGON": "POLYGON_RPC_URL",
    "ETHEREUM": "MAINNET_RPC_URL",
    "ARBITRUM": "ARBITRUM_RPC_URL",
    "KOVAN": "KOVAN_RPC_URL",
    "MUMBAI": "MUMBAI_RPC_URL"
}

# Determine the correct argument to pass based on the RPC value
rpc_arg = {rpc_url_args[RPC]: RPC_URL}

aave_client = AaveClient(WALLET_ADDRESS=os.getenv('WALLET_ADDRESS'),
                                PRIVATE_WALLET_KEY=os.getenv('PRIVATE_WALLET_KEY'),
                                GAS_STRATEGY="medium",
                                **rpc_arg)

# Get the lending pool smart contract:
lending_pool = aave_client.get_lending_pool()

#  Find deposit and borrow assets

In [8]:
assets = aave_client.get_protocol_data("getAllReservesTokens")

assets_symbols = ['DAI', 'USDC', 'USDT']

filtered_assets = [asset for asset in assets if asset[0] in assets_symbols]

df = pd.DataFrame(columns=['symbol', 'currentLiquidityRate', 'currentVariableBorrowRate', 'lastUpdateTimestamp'])

# Iterate over the filtered assets and call the desired functions
for symbol, asset_address in filtered_assets:
    # Call getReserveData for each asset
    reserve_data = aave_client.get_pool_data(lending_pool, "getReserveData", asset_address)
    # Extract the required data
    current_liquidity_rate = reserve_data['currentLiquidityRate'] * 100
    current_variable_borrow_rate = reserve_data['currentVariableBorrowRate'] * 100
    last_update_timestamp = pd.to_datetime(reserve_data['lastUpdateTimestamp'], unit='s')
    
    # Append the data to the DataFrame
    df.loc[asset_address] = [symbol, current_liquidity_rate, current_variable_borrow_rate, last_update_timestamp]

# Find the asset address with the maximum currentLiquidityRate
deposit_asset = df['currentLiquidityRate'].idxmax()

# Find the asset address with the smallest currentVariableBorrowRate
borrow_asset = df['currentVariableBorrowRate'].idxmin()

deposit_apy = df.loc[deposit_asset]['currentLiquidityRate']
borrow_apy = df.loc[borrow_asset]['currentVariableBorrowRate']

print("Supplied asset is: ", deposit_asset)
print("Borrowed asset is: ", borrow_asset)

Supplied asset is:  0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174
Borrowed asset is:  0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063


#  Find current wallet positions

In [6]:
def interpret_user_reserve_data(user_reserve_data):
    # Extract the relevant value from the tuple
    user_reserve_value = user_reserve_data[0]

    # Convert the user_reserve_value to binary representation
    binary_data = bin(user_reserve_value)[2:]

    # Pad the binary data with leading zeros to ensure it has a length of 32 bits
    binary_data = binary_data.zfill(32)

    # Split the binary data into pairs of bits
    bit_pairs = [binary_data[i:i+2] for i in range(0, len(binary_data), 2)]

    # Initialize empty lists to store the indices of borrowed and collateral assets
    borrowed_indices = []
    collateral_indices = []

    # Iterate over the bit pairs, starting from the right
    for i in range(len(bit_pairs)-1, -1, -1):
        pair = bit_pairs[i]
        asset_index = len(bit_pairs) - i - 1

        if pair == "01":
            borrowed_indices.append(asset_index)
        elif pair == "11":
            collateral_indices.append(asset_index)
            borrowed_indices.append(asset_index)
        elif pair == "10":
            collateral_indices.append(asset_index)

    return borrowed_indices, collateral_indices

def get_user_config():
    return aave_client.get_pool_data(lending_pool, "getUserConfiguration", user_address)

def get_reserves_list():
    return aave_client.get_pool_data(lending_pool, "getReservesList")



In [7]:
user_address = os.getenv('WALLET_ADDRESS')

user_config = get_user_config()
borrowed_indices, collateral_indices = interpret_user_reserve_data(user_config)
reserves_assets_list = get_reserves_list()

# verify that borrowed and collateral assets are unique
if len(borrowed_indices) > 1:
    print("More than 1 asset are being borrowed")

if len(collateral_indices) > 1:
    print("More than 1 asset are being used as collateral")


print("Supplied asset is: ", reserves_assets_list[collateral_indices[0]])
print("Borrowed asset is: ", reserves_assets_list[borrowed_indices[0]])

Supplied asset is:  0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174
Borrowed asset is:  0xc2132D05D31c914a87C6611C10748AEb04B58e8F


#  Compare current with target positions

In [ ]:
deposit_asset == reserves_assets_list[collateral_indices[0]]

borrow_asset == reserves_assets_list[borrowed_indices[0]]

#  Switch positions

In [3]:

#deposit_apy=0.0165
#reward_apr=0.0546

#borrow_stable_apy=None
#borrow_variable_apy=0.0562

# in year
time_1=1

# times to deosit and borrow
loop_count=4

# Token Price
token_price=72

ltv=ltv_list[0]

In [4]:
class aave_recursive_borrowing_strategy:
    def __init__(self, DepositAmount:float, TokenPrice:float, LTV:float, DepostiAPY:float, \
                 RewardAPY:float, Time:float, BorrowAPY:float):
        self.DepositAmount=DepositAmount
        self.TokenPrice=TokenPrice
        self.LTV=LTV
        self.DepostiAPY=DepostiAPY
        self.RewardAPY=RewardAPY
        self.Time=Time
        self.BorrowAPY=BorrowAPY
    
    def value(self):
        self.DepositValue = self.DepositAmount * self.TokenPrice
        self.BorrowValue = self.DepositValue * self.LTV
        self.BorrowAmount = self.BorrowValue / self.TokenPrice
        
        return self.DepositValue, self.BorrowValue, self.BorrowAmount
    
    def income(self):
        self.DepositIncome = self.DepositAmount * self.TokenPrice * self.DepostiAPY * self.Time
        self.RewardIncome = self.DepositAmount * self.TokenPrice * self.RewardAPY * self.Time
        self.TotalIncome = self.DepositIncome + self.RewardIncome
        
        return self.DepositIncome, self.RewardIncome, self.TotalIncome
    
    # Interest Rate
    def cost(self):
        self.BorrowCost = self.BorrowAmount * self.TokenPrice * self.BorrowAPY * self.Time
        
        return self.BorrowCost
    
    def net_income(self):
        self.NetIncome = self.TotalIncome - self.BorrowCost
        self.Net_Income_Token_Amount = self.NetIncome/self.TokenPrice
        
        return self.NetIncome, self.Net_Income_Token_Amount
    
    def apy(self):
        self.Dep_Borr_APY=self.NetIncome/self.DepositValue/self.Time
        
        return self.Dep_Borr_APY
    


In [5]:
# Initializing
borrow_amt=initial_token_amt
sum_net_income_token_amt=0
sum_borrow_amt=0

col_header_list=['Deposit Amount', 'Borrow Amount', 'Net Income (/Token)', 'APY']
dep_borr_record_pd = pd.DataFrame(None, index=list(range(1,loop_count+1)), columns=col_header_list)


In [6]:

for i in range(loop_count):
    deposit_amt=borrow_amt
    strategy=aave_recursive_borrowing_strategy(DepositAmount=deposit_amt, TokenPrice=token_price, LTV =ltv, DepostiAPY=deposit_apy, \
                     RewardAPY=reward_apr, Time=time_1, BorrowAPY=borrow_variable_apy)

    value=strategy.value()
    income=strategy.income()
    cost=strategy.cost()
    net_income=strategy.net_income()
    apy=strategy.apy()
    
    # TODO: change borrow_amt to AAVE client txn receipt output
    borrow_amt=value[2]
    net_income_in_token=net_income[1]

    sum_borrow_amt=sum_borrow_amt+borrow_amt
    sum_net_income_token_amt=sum_net_income_token_amt+net_income[1]
    
    # Result for each time, write to DataFrame
    data_list=[deposit_amt, borrow_amt, net_income_in_token, apy]
    dep_borr_record_pd.iloc[i]=data_list

collateral_ratio=sum_borrow_amt/initial_token_amt
apy_after_loop=sum_net_income_token_amt/initial_token_amt/time_1
print()
print('Total Borrow Amount =',sum_borrow_amt)
print('Total Net Income (/Token) =',sum_net_income_token_amt)

print('Collateral Ratio =', collateral_ratio)
print('APY after Loop = ',apy_after_loop)

print()
print('Result for Each Time')
print(dep_borr_record_pd)


Total Borrow Amount = 93.75
Total Net Income (/Token) = 8.062500000000002
Collateral Ratio = 0.9375
APY after Loop =  0.08062500000000002

Result for Each Time
  Deposit Amount Borrow Amount Net Income (/Token)    APY
1            100          50.0                 4.3  0.043
2           50.0          25.0                2.15  0.043
3           25.0          12.5               1.075  0.043
4           12.5          6.25              0.5375  0.043
